In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents.peak_detection as pkd
import numpy as np
import time

/home/aurelien/Documents/Rust/spikeinterface_rust/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": True, "chunk_duration":"60s"}

In [4]:
rec, sorting = si.generate_ground_truth_recording(durations=[60],num_channels=128,sampling_frequency=25000.0,num_units= 100, seed=42)
rec = rec.save_to_memory(**job_kwargs)
_ = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

noise_level (workers: 20 threads): 100%|██████████| 20/20 [00:00<00:00, 231.96it/s]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


In [5]:
peaks_numba = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)
peaks_rust = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

print(len(peaks_numba))
print(len(peaks_rust))

detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]
detect peaks (locally_exclusive) (workers: 1 threads):   0%|          | 0/1 [00:00<?, ?it/s]

At sample 34, channel 26: min_current_ch = -35.73156
Current value: -35.73156
Minimum in j=90 : -58.95111
Compared neighbours: true false
Possible peaks: true false


detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]

77722
91557


In [ ]:
rec.get_traces(start_frame=30, end_frame=35,channel_ids=['26'])
rec.get_traces(start_frame=28, end_frame=35,channel_ids=['90'])

array([[-40.118492],
       [-26.00287 ],
       [-18.66333 ],
       [-32.78183 ],
       [-18.216942]], dtype=float32)

In [7]:
rec.get_traces(start_frame=30, end_frame=35,channel_ids=['24', '25', '26', '27', '28', '88', '89', '90', '91', '92'])

array([[-24.153301  , -29.441748  , -31.786     , -30.569645  ,
        -18.573526  , -25.680199  , -30.641273  , -40.118492  ,
        -32.587585  , -24.126806  ],
       [-16.213049  , -30.36327   , -28.192102  , -32.753582  ,
        -11.899261  , -22.208588  , -30.556465  , -26.00287   ,
        -34.82797   , -11.720132  ],
       [-20.116875  , -26.192884  , -35.73156   ,  -8.630941  ,
        -19.453423  ,  -8.925003  , -13.106163  , -18.66333   ,
        -15.690507  , -14.995763  ],
       [-14.606274  , -14.099154  , -17.21137   , -16.566515  ,
        -20.083141  , -13.098045  , -22.52763   , -32.78183   ,
         -6.6786394 , -17.38895   ],
       [ -6.182972  , -10.682719  , -12.400566  , -21.172035  ,
        -12.938122  ,  -4.6926627 , -19.282627  , -18.216942  ,
         -6.110552  ,  -0.70188046]], dtype=float32)

In [8]:
print(peaks_numba[-10:-1])
print(peaks_rust[-10:-1])

[(1499805, 107,  -74.8041153 , 0) (1499809,  12,  -42.23897552, 0)
 (1499817,  88, -120.50748444, 0) (1499826,  26,  -84.51519775, 0)
 (1499847,   2,  -81.30005646, 0) (1499855, 108,  -55.4861145 , 0)
 (1499883,  81, -167.32948303, 0) (1499889,  56,  -39.03798676, 0)
 (1499987,  90,  -70.16048431, 0)]
[(1499809,  12,  -42.23897552, 0) (1499817,  88, -120.50748444, 0)
 (1499826,  26,  -84.51519775, 0) (1499847,   2,  -81.30005646, 0)
 (1499855, 108,  -55.4861145 , 0) (1499883,  81, -167.32948303, 0)
 (1499889,  56,  -39.03798676, 0) (1499892,  23,  -30.35818672, 0)
 (1499987,  90,  -70.16048431, 0)]


In [9]:
np.setdiff1d(peaks_rust, peaks_numba)

array([(     32, 26, -35.73155975, 0), (     33, 90, -32.78182983, 0),
       (    155,  3, -34.04723358, 0), ...,
       (1499743, 35, -37.73421097, 0), (1499744, 38, -30.66900444, 0),
       (1499892, 23, -30.35818672, 0)],
      shape=(13835,), dtype=[('sample_index', '<i8'), ('channel_index', '<i8'), ('amplitude', '<f8'), ('segment_index', '<i8')])

In [10]:
%timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

detect peaks (locally_exclusive) (workers: 1 threads):   0%|          | 0/1 [00:00<?, ?it/s]

At sample 34, channel 26: min_current_ch = -35.73156
Current value: -35.73156
Minimum in j=90 : -58.95111
Compared neighbours: true false
Possible peaks: true false


detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]
detect peaks (locally_exclusive) (workers: 1 threads):   0%|          | 0/1 [00:00<?, ?it/s]

At sample 34, channel 26: min_current_ch = -35.73156
Current value: -35.73156
Minimum in j=90 : -58.95111
Compared neighbours: true false
Possible peaks: true false


detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]
detect peaks (locally_exclusive) (workers: 1 threads):   0%|          | 0/1 [00:00<?, ?it/s]

...

At sample 34, channel 26: min_current_ch = -35.73156
Current value: -35.73156
Minimum in j=90 : -58.95111
Compared neighbours: true false
Possible peaks: true false


detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]
detect peaks (locally_exclusive) (workers: 1 threads):   0%|          | 0/1 [00:00<?, ?it/s]

At sample 34, channel 26: min_current_ch = -35.73156
Current value: -35.73156
Minimum in j=90 : -58.95111
Compared neighbours: true false
Possible peaks: true false


detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]

606 ms ± 4.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.49s/it]
detect peaks (locally_exclusive) (workers: 1 threads): 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]

1.46 s ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

Chunk duration :500 ms
At sample 34, channel 26: min_current_ch = -35.73156
Current value: -35.73156
Minimum in j=90 : -58.95111
Compared neighbours: true false
Possible peaks: true false
At sample 34, channel 26: min_current_ch = -47.72475
Current value: -38.97137
Minimum in j=90 : -47.110756
Compared neighbours: true false
Possible peaks: false false
At sample 34, channel 90: min_current_ch = -47.110756
Current value: -43.174416
Minimum in j=90 : -47.110756
Compared neighbours: true true
Possible peaks: false false
At sample 34, channel 26: min_current_ch = -63.40043
Current value: -43.539783
Minimum in j=90 : -57.7139
Compared neighbours: true false
Possible peaks: false false
At sample 34, channel 90: min_current_ch = -57.7139
Current value: -55.261337
Minimum in j=90 : -57.7139
Compared neighbours: true true
Possible peaks: false false
At sample 34, channel 26: min_current_ch = -95.21304
Current value: -52.061306
Minimum in j=90 : -88.77246
Compared neighbours: true false
Possible

In [13]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

Chunk duration :500 ms


KeyboardInterrupt: 